In [1]:
from collections import defaultdict
import json
import statistics

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

%matplotlib notebook

In [2]:
matches = pd.read_csv("https://raw.githubusercontent.com/avgupta456/statbotics-csvs/main/matches.csv")

In [3]:
for year in range(2002, 2021):
    sykes_matches = pd.read_csv("sykes/" + str(year) + ".csv")
    
    sykes_acc, sykes_mse, sykes_count = 0, 0, 0
    for _, match in sykes_matches.iterrows():
        pred = match["Red win likelihood"]
        if type(pred) != float:
            pred = float(pred.split("%")[0]) / 100
        winner = match["Red victory"]
        
        if not (pred >= 0 and winner >= 0):
            continue
        
        if pred >= 0.5 and winner >= 1:
            sykes_acc += 1
        if pred < 0.5 and winner <= 0:
            sykes_acc += 1
        sykes_mse += (pred - winner) ** 2
        
        sykes_count += 1
        
    sykes_acc = round(sykes_acc / sykes_count, 4)
    sykes_mse = round(sykes_mse / sykes_count, 4)
        
    sb_matches = matches[matches.year == year]
    
    sb_acc, sb_mse, sb_count = 0, 0, 0
    for _, match in sb_matches.iterrows():
        pred = match["elo_win_prob"]
        winner = 1 if match["winner"] == "red" else (0 if match["winner"] == "blue" else 0.5)
        
        if not (pred >= 0 and winner >= 0):
            continue
        
        if pred >= 0.5 and winner >= 1:
            sb_acc += 1
        if pred < 0.5 and winner <= 0:
            sb_acc += 1
        sb_mse += (pred - winner) ** 2
        sb_count += 1
        
    sb_acc = round(sb_acc / sb_count, 4)
    sb_mse = round(sb_mse / sb_count, 4)

    print(
        year, "\t", 
        sykes_count, "\t", 
        "{:.4f}".format(sykes_acc), 
        "\t", "{:.4f}".format(sykes_mse), 
        "\t\t", sb_count, 
        "\t", "{:.4f}".format(sb_acc), 
        "\t", "{:.4f}".format(sb_mse)
    )

2002 	 2197 	 0.5626 	 0.2355 		 2197 	 0.5589 	 0.2364
2003 	 3173 	 0.6193 	 0.2229 		 3173 	 0.6199 	 0.2214
2004 	 3299 	 0.6156 	 0.2133 		 3198 	 0.6316 	 0.2106
2005 	 2068 	 0.6620 	 0.2085 		 2059 	 0.6610 	 0.2083
2006 	 3283 	 0.6643 	 0.2031 		 3283 	 0.6616 	 0.2035
2007 	 3564 	 0.6181 	 0.2094 		 3564 	 0.6181 	 0.2090
2008 	 4038 	 0.6672 	 0.1953 		 4038 	 0.6677 	 0.1950
2009 	 4658 	 0.6816 	 0.1956 		 4658 	 0.6836 	 0.1958
2010 	 5567 	 0.6248 	 0.1762 		 5567 	 0.6246 	 0.1764
2011 	 6312 	 0.7115 	 0.1679 		 6312 	 0.7113 	 0.1680
2012 	 7707 	 0.6852 	 0.1874 		 7707 	 0.6851 	 0.1876
2013 	 8242 	 0.7265 	 0.1798 		 8242 	 0.7263 	 0.1795
2014 	 10663 	 0.7023 	 0.1890 		 10663 	 0.7022 	 0.1890
2015 	 11810 	 0.7016 	 0.1857 		 11810 	 0.7027 	 0.1855
2016 	 13286 	 0.7019 	 0.1867 		 13286 	 0.7023 	 0.1864
2017 	 15429 	 0.6488 	 0.2094 		 15429 	 0.6492 	 0.2092
2018 	 16930 	 0.7388 	 0.1726 		 16930 	 0.7395 	 0.1722
2019 	 18022 	 0.7017 	 0.1829 		 1802